# SuSIE results post process

This notebook is to post-process the susie results into different text file


In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb susie_to_tsv \
    --cwd output/test --rds_path `ls output/test/cache/*rds | head ` --region-list <(head -50  ./dlpfc_region_list) --container containers/stephenslab.sif 

In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb susie_to_tsv \
    --cwd output/ADGWAS_finemapping_extracted/Bellenguez/ --rds_path `ls GWAS_Finemapping_Results/Bellenguez/ADGWAS2022*rds ` \
    --region-list ~/1300_hg38_EUR_LD_blocks_orig.tsv \
    --container containers/stephenslab.sif 

sos run pipeline/SuSiE_post_processing.ipynb susie_tsv_collapse \
    --cwd output/ADGWAS_finemapping_extracted --tsv_path `ls output/ADGWAS_finemapping_extracted/*lbf.tsv` \
    --container containers/stephenslab.sif 

In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_haQTL_pos --rds_path `ls output/f_susie_tad_haQTL_pos/cache/*rds ` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif -s build

In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_meQTL_pos_selected/ --rds_path `ls output/f_susie_tad_meQTL_pos_selected//cache/*1204*rds ` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif -s build

In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_meQTL_pos/ --rds_path `ls output/f_susie_tad_meQTL_pos//cache/*rds ` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif -s build

In [ ]:
[global]
import glob
import pandas as pd


# A region list file documenting the chr_pos_ref_alt of a susie_object
parameter: cwd = path("output")
parameter: name = "demo"

## Path to work directory where output locates
## Containers that contains the necessary packages
parameter: container = ""
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 20
parameter: windows = 1000000
# use this function to edit memory string for PLINK input
from sos.utils import expand_size

In [1]:
[susie_to_tsv]
# Input
# For complete susie, region_list or tad_list, for susie_rss , LD region list 
parameter: region_list = path
region_tbl = pd.read_csv(region_list,sep = "\t")
parameter: rds_path = paths
input: rds_path, group_by = 1
output: f"{cwd}/{_input:bn}.lbf.tsv",f"{cwd}/{_input:bn}.variant.tsv",f"{cwd}/{_input:bn}.effect.tsv"
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
R: expand = '${ }', stdout = f"{_output[0]:nn}.stdout", stderr = f"{_output[0]:nn}.stderr", container = container
    library("dplyr")
    library("tibble")
    library("purrr")
    library("tidyr")
    library("readr")
    library("stringr")
    library("susieR")
    extract_lbf = function(susie_obj){
    
    if("variants" %in% names(susie_obj) ){
    ss_bf = tibble(snps = susie_obj$variants, cs_index = ifelse(is.null(susie_obj$sets$cs_index), 0, paste0(susie_obj$sets$cs_index,collapse =",")),names = "${f'{_input:br}'.split('.')[-4]}")
    }
      else 
      {
    ss_bf = tibble(snps = susie_obj$variable_name, cs_index = ifelse(is.null(susie_obj$sets$cs_index), 0, paste0(susie_obj$sets$cs_index,collapse =",")),names = "${_input:bnnn}")
     }
    
    ss_bf = ss_bf%>%cbind(susie_obj$lbf_variable%>%t)%>%as_tibble()
    
    return(ss_bf)
    }
    
    extract_variants_pip = function(susie_obj,region_list){
    susie_tb = tibble( variants =  names(susie_obj$pip)[which( susie_obj$pip >= 0)],
                           snps_index = which(( susie_obj$pip >= 0))) %>%
        mutate(chromosome = map_chr(variants, ~read.table(text = .x, sep = ":")$V1%>%str_replace("chr","") ),
                position  = map_chr(variants, ~read.table(text = .x, sep = ":")$V2  ),
                ref = map_chr(position , ~read.table(text = .x, sep = "_",colClasses = "character")$V2  ),
                alt = map_chr(position , ~read.table(text = .x, sep = "_",colClasses = "character")$V3  ),
                position  = map_dbl(position , ~read.table(text = .x, sep = "_",as.is = T)$V1  )
                             )
      susie_tb = susie_tb%>%mutate(cs_order =(map(susie_tb$snps_index , ~tryCatch(which(pmap(list( a= susie_obj$sets$cs) , function(a) .x %in% a )%>%unlist()), error = function(e) return(0) )  ))%>%as.character%>%str_replace("integer\\(0\\)","0"),
                         cs_id = map_chr(cs_order,~ifelse(.x =="0", "None" ,names(susie_obj$sets$cs)[.x%>%str_split(":")%>%unlist%>%as.numeric] ) ),
                         log10_base_factor = map_chr(snps_index,~paste0( susie_obj$lbf_variable[,.x],  collapse = ";")),
                         pip = susie_obj$pip,
                         posterior_mean = coef.susie(susie_obj)[-1],
                         posterior_sd = susie_get_posterior_sd(susie_obj),
                         z = posterior_mean/posterior_sd)
    
          susie_tb =  susie_tb%>%mutate(  molecular_trait_id = region_list$molecular_trait_id,
                             finemapped_region_start = region_list$finemapped_region_start,
                             finemapped_region_end = region_list$finemapped_region_end)
          return(susie_tb)    }
          
        

     extract_effect_pip = function(susie_obj,region_list,susie_tb){
      result_tb =  tibble(phenotype = susie_obj$name,
        V = susie_obj$V,effect_id = paste0("L",1:length(V) ) ,
        cs_log10bf = susie_obj$lbf)
        if(is.null(susie_obj$sets$cs)){
            cs_min_r2 = cs_avg_r2 =  coverage =  0 
            cs = "None"} else {         cs = map_chr(susie_obj$sets$cs[result_tb$effect_id],~susie_tb$variants[.x]%>%paste0(collapse = ";"))
        coverage = map(result_tb$effect_id, ~susie_obj$sets$coverage[which(names(susie_obj$sets$cs) == .x )])%>%as.numeric%>%replace_na(0)
        cs_min_r2  = (susie_obj$sets$purity[result_tb$effect_id,1])%>%as.numeric%>%replace_na(0)  
        cs_avg_r2  = (susie_obj$sets$purity[result_tb$effect_id,2])%>%as.numeric%>%replace_na(0) }
        result_tb = result_tb%>%mutate(cs_min_r2 = cs_min_r2,cs_avg_r2 = cs_avg_r2 ,coverage = coverage%>%unlist,cs = cs )            
      return(result_tb)
      }
       
  
    susie_obj = readRDS("${_input:a}")
    if("variants" %in% names(susie_obj) ){susie_obj$variants = susie_obj$variants%>%str_replace("_",":")}
    if(is.null(names(susie_obj$pip ))){names(susie_obj$pip) = susie_obj$variants}
    lbf = extract_lbf(susie_obj)
    region_list = read_delim("${region_list}","\t")
    if(ncol(region_list) == 3 ){   region_list =  region_list%>%mutate(`#chr` = `#chr`%>%str_remove_all(" ") , ID = paste0(`#chr`,"_",start,"_",end) ) } # LD_list 
    if(region_list$start[1] - region_list$end[1]  == -1 ){ 
        region_list = region_list%>%mutate( start = start - ${windows} ,end = start +${windows}) # region_list for fix cis windows  
          } 
      if("gene_id" %in% colnames(region_list)){region_list = region_list%>%mutate(ID = gene_id)  } # region_list for gene
    region_list = region_list%>%select(molecular_trait_id = ID, chromosome  = `#chr`,finemapped_region_start = start ,finemapped_region_end = end)  # Formatting
    region_list = region_list%>%filter(molecular_trait_id == "${f'{_input:br}'.split('.')[-4]}")
    variants_pip = extract_variants_pip( susie_obj , region_list)
    effect_pip = extract_effect_pip( susie_obj , region_list,variants_pip)
    lbf%>%write_delim("${_output[0]}","\t")
    variants_pip%>%write_delim("${_output[1]}","\t")
    effect_pip%>%write_delim("${_output[2]}","\t")

In [ ]:
[fsusie_to_tsv]
# Input
# For complete susie, region_list or tad_list, for susie_rss , LD region list 
parameter: region_list = path
region_tbl = pd.read_csv(region_list,sep = "\t")
parameter: rds_path = paths
input: rds_path, group_by = 1
output: f"{cwd}/{_input:bn}.variant.tsv"
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
R: expand = '${ }', stdout = f"{_output[0]:nn}.stdout", stderr = f"{_output[0]:nn}.stderr", container = container
    library("dplyr")
    library("tibble")
    library("purrr")
    library("tidyr")
    library("readr")
    library("stringr")
    library("susieR")

    extract_variants_pip = function(susie_obj,region_list){
        susie_tb = tibble( variants =  names(susie_obj$csd_X),
                               snps_index = which(( susie_obj$pip >= 0))) %>%
            mutate(chromosome = map_chr(variants, ~read.table(text = .x, sep = ":")$V1%>%str_replace("chr","") ),
                    position  = map_chr(variants, ~read.table(text = .x, sep = ":")$V2  ),
                    ref = map_chr(position , ~read.table(text = .x, sep = "_",colClasses = "character")$V2  ),
                    alt = map_chr(position , ~read.table(text = .x, sep = "_",colClasses = "character")$V3  ),
                    position  = map_dbl(position , ~read.table(text = .x, sep = "_",as.is = T)$V1  )
                                 )
          susie_tb = susie_tb%>%mutate(cs_order =(map(susie_tb$snps_index , ~tryCatch(which(pmap(list( a= susie_obj$cs) , function(a) .x %in% a )%>%unlist()), error = function(e) return(0) )  ))%>%as.character%>%str_replace("integer\\(0\\)","0"),
                             pip = susie_obj$pip)
          susie_tb =  susie_tb%>%mutate(  molecular_trait_id = region_list$tad_index,
                                 finemapped_region_start = region_list$start,
                                 finemapped_region_end = region_list$end)
    susie_tb = cbind(susie_tb,Reduce(cbind, susie_obj$lBF)%>%as.tibble%>%`colnames<-`(1:length(susie_obj$lBF)))
          return(susie_tb)    }
    susie_obj = readRDS("${_input:a}")
    region_list = read_delim("${region_list}","\t")
    region_list = region_list%>%filter(tad_index == "${f'{_input:br}'.split('.')[-4]}")
    variants_pip = extract_variants_pip( susie_obj , region_list)
    print(paste0("fsusie run time is ", round(susie_obj$runtime[[3]]/60),"min"))
    variants_pip%>%write_delim("${_output}","\t")

In [ ]:
[susie_tsv_collapse]
parameter: tsv_path = paths # TSV needs have the name ends with  *.chr1_2_3.unisusie(_rss).lbf.tsv
tsv_list  = pd.DataFrame({"lbf_path" : [str(x) for x in tsv_path]})
chromosome = list(set([f'{x.split(".")[-5].split("_")[0].replace("chr","")}'  for x in tsv_list.lbf_path ])) ## Add chr if there is no chr prefix. This is to accomodata chr XY and M
input: tsv_path, for_each = "chromosome"
output: f'{cwd}/{_input[0]:bnnnnnnn}.chr{_chromosome}.unisusie_rss.lbf.tsv'
bash: expand = '${ }', stdout = f"{_output}.stdout", stderr = f"{_output}.stderr", container = container
        head -1 ${_input[0]} > ${_output}
        cat ${_input[0]:d}/*.chr${_chromosome}_*lbf.tsv | grep -v cs_index >> ${_output}
        

In [ ]:
[susie_overlap_plot]
parameter: tsv_path = paths # TSV needs have the name ends with  *.chr1_2_3.unisusie(_rss).lbf.tsv
tsv_list  = pd.DataFrame({"lbf_path" : [str(x) for x in tsv_path]})
chromosome = list(set([f'{x.split(".")[-5].split("_")[0].replace("chr","")}'  for x in tsv_list.lbf_path ])) ## Add chr if there is no chr prefix. This is to accomodata chr XY and M
input: tsv_path, for_each = "chromosome"
output: f'{cwd}/{_input[0]:bnnnnnnn}.chr{_chromosome}.unisusie_rss.lbf.tsv'
bash: expand = '${ }', stdout = f"{_output}.stdout", stderr = f"{_output}.stderr", container = container
        head -1 ${_input[0]} > ${_output}
        cat ${_input[0]:d}/*.chr${_chromosome}_*lbf.tsv | grep -v cs_index >> ${_output}